In [1]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:3 https://download.docker.com/linux/ubuntu focal InRelease                 
Hit:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease 
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]        
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Hit:8 http://security.ubuntu.com/ubuntu focal-security InRelease               
Get:9 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]          
Get:10 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [4726 B]
Fetched 153 kB in 1s (172 kB/s)m0m
Reading package lists... Done
Building dependency tree       
Reading state information... Done
13 packages can be upgraded. 

In [2]:
!pip install -q vllm fastapi uvicorn ngrok pyngrok nest_asyncio

In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from vllm import LLM, SamplingParams
import uvicorn
import os
import torch
from transformers import AutoTokenizer
from pyngrok import ngrok
import nest_asyncio
from starlette.responses import StreamingResponse
import asyncio

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
ngrok.set_auth_token('2hgvs8djD2xuTeO4svNzxiIM7G4_7qyy1Jo94tfQ3Acvoba1t')

In [8]:
BASE_MODEL = "google/gemma-2b-it"
FINTUNED_MODEL = "bradmin/gemma-2b-persona-instruct"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

tensor_parallel_size = int(os.environ.get("DEVICES", "1"))

llm = LLM(
    model=FINTUNED_MODEL,
    dtype=torch.float16,
    # kv_cache_dtype="fp8",
    tensor_parallel_size=tensor_parallel_size,
    )

OutOfMemoryError: CUDA out of memory. Tried to allocate 1000.00 MiB. GPU 

In [ ]:
nest_asyncio.apply()

app = FastAPI()

formatted_prompt = """<bos><start_of_turn>user
당신은 유저의 대답을 듣고 그를 진정시킨 후, 그의 이야기가 얼마나 현실성 없는 이야기인지 대답해주는 모델입니다. 다양한 방식으로 그를 설득해 회사에 남아있고 그의 목표를 나중에 시도하라고 이야기 해주세요.

{}<end_of_turn>
<start_of_turn>model\n
"""

class CompletionRequest(BaseModel):
    prompt: str
    max_tokens: int
    min_tokens: int
    repetition_penalty: float
    temperature: float
    top_k: int
    top_p: float

@app.post("/v1/completions")
async def completions(request: CompletionRequest):
  sampling_params = SamplingParams(
      max_tokens=request.max_tokens,
      min_tokens=request.min_tokens,
      repetition_penalty=request.repetition_penalty,
      temperature=request.temperature,
      top_k=request.top_k,
      top_p=request.top_p,
      )
  prompt = formatted_prompt.format(request.prompt)
  print(prompt)
  response = llm.generate([prompt], sampling_params)
  ans = response[0].outputs[0].text
  return ans

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [4995]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://8fe3-54-226-179-194.ngrok-free.app" -> "http://localhost:8000"
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
INFO:     54.226.179.194:0 - "POST /v1/completions HTTP/1.1" 422 Unprocessable Entity
